Get everything set up as it was for the retrain step

In [1]:
import torch
import my_vision
from torchvision import datasets, models, transforms

data_transforms = {
    'train': transforms.Compose([
        transforms.ToTensor(),
        transforms.RandomHorizontalFlip(),        
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

data_dir = 'images/clean_image_data'
image_dataset = my_vision.classification_dataset("training_data.csv",data_dir,data_transforms['test'])



dataloader =torch.utils.data.DataLoader(image_dataset, batch_size=4,
                                             shuffle=True, num_workers=4)
              
dataset_sizes =  len(image_dataset) 

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = torch.load('model_evaluation/feature/best_weights.pt')



Assume last layer is avgpool as it would be with resnet 18

In [2]:

layer = model._modules.get('avgpool')

Get the image into the right state

In [3]:
import image_processor
pyImage = image_processor.fullPreProcess_Image(filePath='images/clean_image_data/0a1baaa8-4556-4e07-a486-599c05cce76c.jpg').unsqueeze(0)

 # 3. Create a vector of zeros that will hold our feature vector
    #    The 'avgpool' layer has an output size of 512
my_embedding = torch.zeros(512)
# 4. Define a function that will copy the output of a layer
def copy_data(m, i, o):
    my_embedding.copy_(o.data.reshape(o.data.size(1)))
    # 5. Attach that function to our selected layer

h = layer.register_forward_hook(copy_data)

path processing: images/clean_image_data/0a1baaa8-4556-4e07-a486-599c05cce76c.jpg


Run model on our image

In [7]:
model(pyImage.to(device))

# 7. Detach our copy function from the layer
h.remove()

print(my_embedding)

RuntimeError: output with shape [512] doesn't match the broadcast shape [1, 2048, 1, 512]